# Finding the Optimal Policy with Linear Programming

In [ ]:
from enum import IntEnum


# MDP from the previous lecture.
class State(IntEnum):
    S0 = 0
    W = 1
    M = 2
    WM = 3
    SUCCESS = 4
    ABANDON = 5


class Action(IntEnum):
    ASK_WEATHER = 0
    ASK_MOOD = 1
    ASK_BOTH = 2
    RECOMMEND = 3


TERMINAL_STATES = {State.SUCCESS, State.ABANDON}
NON_TERMINAL_STATES = [s for s in State if s not in TERMINAL_STATES]

AVAILABLE_ACTIONS = {
    State.S0: [Action.ASK_WEATHER, Action.ASK_MOOD, Action.ASK_BOTH],
    State.W: [Action.ASK_MOOD],
    State.M: [Action.ASK_WEATHER],
    State.WM: [Action.RECOMMEND],
    State.SUCCESS: [],
    State.ABANDON: [],
}

TRANSITIONS = {
    (State.S0, Action.ASK_WEATHER): {
        State.W: 0.75,
        State.S0: 0.15,
        State.ABANDON: 0.10,
    },
    (State.S0, Action.ASK_MOOD): {
        State.M: 0.70,
        State.S0: 0.20,
        State.ABANDON: 0.10,
    },
    (State.S0, Action.ASK_BOTH): {
        State.WM: 0.50,
        State.W: 0.20,
        State.M: 0.15,
        State.ABANDON: 0.15,
    },
    (State.W, Action.ASK_MOOD): {
        State.WM: 0.80,
        State.W: 0.10,
        State.ABANDON: 0.10,
    },
    (State.M, Action.ASK_WEATHER): {
        State.WM: 0.80,
        State.M: 0.10,
        State.ABANDON: 0.10,
    },
    (State.WM, Action.RECOMMEND): {
        State.SUCCESS: 0.75,
        State.ABANDON: 0.25,
    },
}


def reward(state, action, next_state):
    """Reward is +10 for reaching SUCCESS, 0 otherwise."""
    return 10.0 if next_state == State.SUCCESS else 0.0


In [ ]:
import pyomo.environ as pyo


# Decision variables V(s) for each state
model = pyo.ConcreteModel("Dialogue_MDP_LP")
model.V = pyo.Var(list(State), domain=pyo.Reals)

# Fix terminal state values to 0
model.terminal_success = pyo.Constraint(expr=model.V[State.SUCCESS] == 0)
model.terminal_abandon = pyo.Constraint(expr=model.V[State.ABANDON] == 0)

In [ ]:
# Objective: minimize sum of values over non-terminal states
model.obj = pyo.Objective(
    expr=sum(model.V[s] for s in NON_TERMINAL_STATES), sense=pyo.minimize
)

In [ ]:
# The Bellman Constraint
model.bellman = pyo.ConstraintList()
#gamma = 0.99  # Discount factor
gamma = 1.0  # No discounting

for s in NON_TERMINAL_STATES:
    for a in AVAILABLE_ACTIONS[s]:
        trans = TRANSITIONS[(s, a)]

        # Expected immediate reward + discounted future value
        rhs = sum(
            prob * (reward(s, a, s_next) + gamma * model.V[s_next])
            for s_next, prob in trans.items()
        )

        model.bellman.add(model.V[s] >= rhs)

In [ ]:
solver = pyo.SolverFactory("ipopt")
result = solver.solve(model, tee=False)

print("Optimal Value Function:")
for s in State:
    print(f"  V({s.name}) = {pyo.value(model.V[s]):.4f}")

In [ ]:
# Extract optimal policy
print("Optimal Policy:")
for s in NON_TERMINAL_STATES:
    v_s = pyo.value(model.V[s])
    best_action = None
    best_q = float("-inf")

    action_values = []
    for a in AVAILABLE_ACTIONS[s]:
        trans = TRANSITIONS[(s, a)]
        q_value = sum(
            prob * (reward(s, a, s_next) + gamma * pyo.value(model.V[s_next]))
            for s_next, prob in trans.items()
        )
        action_values.append((a, q_value))
        if q_value > best_q:
            best_q = q_value
            best_action = a

    # Show all Q-values for this state
    q_str = ", ".join(f"{a.name}={q:.3f}" for a, q in action_values)
    print(f"  π({s.name:8}) = {best_action.name:12}  [Q-values: {q_str}]")